In [47]:
import numpy as np
from numpy.random import choice

initialize state

read program, and for each gate:

    calculate matrix operator
    
    apply operator (modify state)

perform multi-shot measurement of all qubits using weighted random technique

In [10]:
single_q_gates = {'x':np.array([[0, 1], [1, 0]]),
                  'h':np.array([[1/np.sqrt(2), 1/np.sqrt(2)], [1/np.sqrt(2), -1/np.sqrt(2)]])}
I = np.identity(2)
CX = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]])
SWAP = [[1, 0, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0], [0, 0, 0, 1]]

In [2]:
def get_ground_state(num_qubits):
    psi = np.zeros_like(np.arange(2**num_qubits))
    psi[0] = 1
    return psi

In [24]:
def get_operator(total_qubits, gate_unitary, target_qubits):
    if gate_unitary in single_q_gates:
        if target_qubits[0] == 0:
            operator = single_q_gates[gate_unitary]
        else:
            operator = np.identity(2)
            for i in range(1, target_qubits[0]):
                operator = np.kron(operator, I)
            operator = np.kron(operator, single_q_gates[gate_unitary])
        for i in range(total_qubits - target_qubits[0] - 1):
            operator = np.kron(operator, I)
        return operator

In [41]:
def run_program(initial_state, program):
    num_qubits = len(initial_state).bit_length()-1
    state = initial_state
    for operation in program:
        gate, target = operation['gate'], operation['target']
        operator = get_operator(num_qubits, gate, target)
        state = np.dot(operator, state)
    return state

In [95]:
my_circuit = [
{ "gate": "x", "target": [0] },
{ 'gate': 'h', 'target': [1] }
]
initial_state = get_ground_state(4)
state = run_program(initial_state, my_circuit)
print(state)

[0.         0.         0.         0.         0.         0.
 0.         0.         0.70710678 0.         0.         0.
 0.70710678 0.         0.         0.        ]


In [96]:
def measure_all(state_vector):
    return "{0:b}".format(choice(range(len(state_vector)), 1, p=np.square(state_vector))[0]).zfill(len(state_vector).bit_length()-1)

In [97]:
def get_counts(state_vector, num_shots):
    counts = {}
    for i in range(num_shots):
        measurement = measure_all(state_vector)
        if measurement not in counts.keys():
            counts[measurement] = 1
        else:
            counts[measurement] += 1 
    return counts

In [98]:
print(get_counts(state, 1000))

{'1000': 496, '1100': 504}
